In [ ]:
import sys
sys.path.append("..")

%load_ext autoreload
%autoreload 1
%aimport src.datasets.joins
%aimport src.datasets.loading.statcan

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from src.datasets.loading import statcan
from src.datasets.loading import ookla
from src.datasets import overlays

import statsmodels as sm
# import statsmodels.stats.weightstats
from scipy.stats import lognorm

In [ ]:
from src.datasets import joins

In [ ]:
popctrs = statcan.boundary('population_centres')

In [ ]:
o = joins.hexagons_popctrs_overlay()

In [ ]:
tiles = ookla.canada_speed_tiles().loc[lambda s:(s.year==2022 ) & (s.conn_type=='fixed')]
tiles = tiles.to_crs(popctrs.crs)

In [ ]:
o_statted = joins.add_simple_stats(o, tiles, 'HEXUID_PCPUID')
o_statted = joins.add_50_10_stats(o_statted, tiles, 'HEXUID_PCPUID')
o_statted = o_statted.merge(statcan.hexagon_data(), on='HEXuid_HEXidu', how='left')

In [ ]:
def breakpoint(value):
    if value == '>75% -  100%':
        return 0.75
    elif value == '>50% - 75%':
        return 0.5
    elif value == '>25% - 50%':
        return 0.25
    elif value == '>0% - 25%':
        return 0.0
    else:
        return 0.0

o_statted['access_level_numerical'] = o_statted.Avail_50_10_Gradient_Dispo.apply(breakpoint)

In [ ]:
for col in o_statted:
    if "kbps" in col:
        o_statted[col] /= 1000
        o_statted.rename(columns={col:col.replace('kbps','Mbps')}, inplace=True)

In [ ]:
def calculate_50_10_level(p50down, p10up):
    p = min(p50down, p10up)

    if p== 0:
        return '0% - Unavailable'
    elif p < 25:
        return '0%-25%'
    elif p < 5:
        return '25%-50%'
    elif p < 75:
        return '50%-75%'
    elif p < 100:
        return  '75%-100%'
    elif p ==100:
        return '100%'
    else:
        return 'Unknown'
    
def ookla_numerical(v):
    if v == '100%':
        return 1.0
    elif v == '75%-100%':
        return 0.75
    elif v == '50%-75%':
        return 0.5
    elif v == '25%-50%':
        return 0.25
    else:
        return 0.0

o_statted['Ookla_Avail_50_10'] = o_statted.apply(lambda s:calculate_50_10_level(s['50_down_percentile'],s['10_up_percentile']), axis=1)
o_statted['Ookla_Avail_50_10_numerical'] = o_statted['Ookla_Avail_50_10'].apply(ookla_numerical)


In [ ]:
o_statted['is_overestimated'] = o_statted['Ookla_Avail_50_10_numerical'] < o_statted['access_level_numerical']

In [ ]:
xmin, ymin, xmax, ymax = popctrs.loc[lambda s:s.PCNAME=="Calgary"].buffer(100000).total_bounds
o_statted.cx[xmin:xmax, ymin:ymax].loc[:,[
    'geometry','HEXUID_PCPUID', 
    'avg_d_Mbps', 'std_d_Mbps', 'min_d_Mbps', '25p_d_Mbps', '50p_d_Mbps', '75p_d_Mbps', 'max_d_Mbps', 
    'avg_u_Mbps', 'std_u_Mbps', 'min_u_Mbps', '25p_u_Mbps', '50p_u_Mbps', '75p_u_Mbps', 'max_u_Mbps', 
    'avg_lat_ms', 
    'tests', 'ave_tests_per_tile',
    'unique_devices', 'ave_devices_per_tile', 
    'num_tiles',
    'SumPop_2016_SommePop', #'SumURD_2016_SommeRH', 'SumTD_2016_SommeTL',
    'Avail_50_10_Gradient_Dispo', 'Ookla_Avail_50_10', 'is_overestimated',
    '50_down_percentile', '10_up_percentile'
]].explore('Ookla_Avail_50_10', cmap=['orange','green','yellow','blue','red','gray'])#, vmin=0, vmax=300)